In [1]:
import cv2
import os
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt
import time

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
def mediapipe_detection(frame, model):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [6]:
def extract_keypoints(results):
    if results.pose_landmarks:
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
    else:
        pose = np.zeros(132)
    
    if results.left_hand_landmarks:
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
    else:
        lh = np.zeros(63)
    
    if results.right_hand_landmarks:
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
    else:
        rh = np.zeros(63)

    return np.concatenate([pose, lh, rh])

In [7]:
#DATA_PATH = os.path.join('MP_data')
actions = np.array(["ambulance", "Danger", "doctor", "Emergency", "help", "hurt", "ShortnessOfBreath"])
no_sequences = 30
sequence_length = 30

In [8]:
#TESTING CAMERA

cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)
        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #print(results.right_hand_landmarks)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()


In [22]:
#Record

cam = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        for sequence in range(no_sequences):
            for frame_no in range(sequence_length):
                
                ret, frame = cam.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_landmarks(image, results)
                
                #print(results.right_hand_landmarks)
                #extract_keypoints(results)
                if frame_no == 0:
                    cv2.putText(image, "Collecting Video for {} Video number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                     cv2.putText(image, "Collecting Video for {} Video number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_no))
                np.save(npy_path, keypoints)
                
                cv2.imshow("Input Sign Language", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cam.release()
                    cv2.destroyAllWindows()

    cam.release()
    cv2.destroyAllWindows()

In [94]:
cam.release()
cv2.destroyAllWindows()     ###OPTIONAL OPTIONAL

In [27]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [28]:
label_map = {label: num for num, label in enumerate(actions)}

In [33]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_no in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_no)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [34]:
x = np.array(sequences)
y = np.array(labels)
y = to_categorical(labels).astype(int)

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.05)

In [36]:
Y_test

array([[0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0,

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [38]:
log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [39]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation="relu", input_shape=(30, 258)))
model.add(LSTM(128, return_sequences=True, activation="relu"))
model.add(LSTM(64, return_sequences=False, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(actions.shape[0], activation="softmax")) #will return a list of probability i.e [0.7, 0.2, 0.1] where sum is 1 and action[0] which is thumbsUp is the prediction with 70% sure

In [40]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [ ]:
model.fit(X_train, Y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
21/21 [==============================] - 4s 203ms/step - loss: 0.0123 - categorical_accuracy: 0.9970
Epoch 2/2000
21/21 [==============================] - 4s 189ms/step - loss: 0.0214 - categorical_accuracy: 0.9925
Epoch 3/2000
21/21 [==============================] - 4s 207ms/step - loss: 0.0174 - categorical_accuracy: 0.9925
Epoch 4/2000
21/21 [==============================] - 4s 176ms/step - loss: 0.0117 - categorical_accuracy: 0.9970
Epoch 5/2000
21/21 [==============================] - 4s 187ms/step - loss: 0.0053 - categorical_accuracy: 0.9985
Epoch 6/2000
21/21 [==============================] - 4s 198ms/step - loss: 0.0046 - categorical_accuracy: 0.9985
Epoch 7/2000
21/21 [==============================] - 4s 202ms/step - loss: 0.0037 - categorical_accuracy: 0.9985
Epoch 8/2000
21/21 [==============================] - 4s 186ms/step - loss: 0.0040 - categorical_accuracy: 0.9985
Epoch 9/2000
21/21 [==============================] - 4s 191ms/step - loss: 0.0036 - cat

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            82688     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 7)                 231       
                                                                 
Total params: 237,383
Trainable params: 237,383
Non-trai

In [43]:
res = model.predict(X_test)

2/2 [==============================] - 0s 33ms/step


In [44]:
np.sum(res)

35.0

In [45]:
actions[np.argmax(res[0])]

'ShortnessOfBreath'

In [46]:
actions[np.argmax(Y_test[0])]

'ShortnessOfBreath'

In [47]:
model.save("action.h5")

In [48]:
res

array([[3.11043108e-10, 1.80173310e-09, 4.04899902e-06, 3.64810404e-11,
        1.04007568e-05, 2.35435092e-11, 9.99985576e-01],
       [1.03766189e-19, 3.21175094e-16, 6.74565384e-13, 3.33037556e-11,
        3.41302073e-12, 1.00000000e+00, 6.85828694e-21],
       [9.99899626e-01, 6.24211118e-19, 4.49306086e-14, 1.14470239e-10,
        1.00374404e-04, 1.30333624e-12, 3.44042125e-16],
       [3.04991121e-09, 6.08678092e-04, 9.97520626e-01, 9.42069892e-06,
        4.40413241e-05, 9.86624286e-07, 1.81626831e-03],
       [2.31010290e-07, 1.13920159e-05, 9.92761374e-01, 8.52830533e-04,
        6.35501416e-03, 2.85708484e-06, 1.64412304e-05],
       [9.99484897e-01, 2.11260976e-16, 1.03696557e-11, 7.67257280e-09,
        5.15080814e-04, 2.09980131e-10, 7.06043568e-13],
       [9.25069710e-09, 1.05223982e-04, 9.98818815e-01, 2.42473725e-05,
        2.36355787e-04, 9.78070602e-07, 8.14234721e-04],
       [2.62570696e-24, 9.99999881e-01, 9.94269698e-08, 3.95131031e-13,
        7.72193329e-20, 2

In [63]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), (0,255,0), -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        
    return output_frame

In [64]:
#PREDICTION AND TESTING

sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-20:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        #cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 36ms/step
Emergency
1/1 [==============================] - 0s 37ms/step
Emergency
1/1 [==============================] - 0s 39ms/step
Emergency
1/1 [==============================] - 0s 37ms/step
Emergency
1/1 [==============================] - 0s 36ms/step
Emergency
1/1 [==============================] - 0s 36ms/step
Emergency
1/1 [==============================] - 0s 36ms/step
Emergency
1/1 [==============================] - 0s 35ms/step
Emergency
1/1 [==============================] - 0s 36ms/step
Emergency
1/1 [==============================] - 0s 37ms/step
Emergency
1/1 [==============================] - 0s 36ms/step
Emergency
1/1 [==============================] - 0s 36ms/step
Emergency
1/1 [==============================] - 0s 41ms/step
Emergency
1/1 [==============================] - 0s 36ms/step
Emergency
1/1 [==============================] - 0s 38ms/step
Emergency
1/1 [==============================] - 0s 36ms/step
Emergency
1/1 [===

In [58]:
cap.release()
cv2.destroyAllWindows()